In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pre-unet/pre-UNet/data.py
/kaggle/input/pre-unet/pre-UNet/train.py
/kaggle/input/pre-unet/pre-UNet/model.py
/kaggle/input/pre-unet/pre-UNet/╫÷╠Γ╫¿╙├.py
/kaggle/input/pre-unet/pre-UNet/utils.py
/kaggle/input/pre-unet/pre-UNet/.idea/.gitignore
/kaggle/input/pre-unet/pre-UNet/.idea/modules.xml
/kaggle/input/pre-unet/pre-UNet/.idea/workspace.xml
/kaggle/input/pre-unet/pre-UNet/.idea/UNet.iml
/kaggle/input/pre-unet/pre-UNet/.idea/misc.xml
/kaggle/input/pre-unet/pre-UNet/.idea/inspectionProfiles/profiles_settings.xml
/kaggle/input/pre-unet/pre-UNet/.idea/inspectionProfiles/Project_Default.xml
/kaggle/input/pre-unet/pre-UNet/dataset/test/labels/SC-HF-NI-14-IM-0001-0116.dcm.png
/kaggle/input/pre-unet/pre-UNet/dataset/test/labels/SC-N-11-IM-0001-0028.dcm.png
/kaggle/input/pre-unet/pre-UNet/dataset/test/labels/SC-HF-I-09-IM-0001-0187.dcm.png
/kaggle/input/pre-unet/pre-UNet/dataset/test/labels/SC-HF-I-09-IM-0001-0118.dcm.png
/kaggle/input/pre-unet/pre-UNet/dataset/test/labels/SC-HYP

In [2]:
# !pip install albumentations==0.4.6

In [3]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
# from model import UNET
# from utils import (
#     load_checkpoint,
#     save_checkpoint,
#     get_loaders,
#     check_accuracy,
#     save_predictions_as_imgs,
#     draw_fig,

# )
import os
from torch import nn
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
# from model import UNET
import torch
import torchvision
# from data import MyDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import torch.nn.functional as F


In [4]:
print(torch.version.cuda)

!nvcc -V
!nvidia-smi 

11.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Mon Apr 18 13:15:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |    

In [5]:
from torch.utils.tensorboard import SummaryWriter

In [6]:
# writer = SummaryWriter("./")

In [7]:
class MyDataset_train(Dataset):
    def __init__(self ,image_dir , mask_dir, edge_dir,transform = None):


        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.edge_dir = edge_dir
        self.transform = transform
        self.images = os.listdir(self.image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.image_dir,self.images[index]) #image图片文件
        mask_path = os.path.join(self.mask_dir,self.images[index]) #mask图片文件 # 按需添加：segment_name.replace('png','jpg')
        edge_path = os.path.join(self.edge_dir,self.images[index])
        
        image = np.array(Image.open(image_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"),dtype=np.float32)
        edge = np.array(Image.open(edge_path).convert("L"),dtype=np.float32)
        mask[mask == 255.0] = 1.0
        edge[edge == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image = image,mask = mask,edge = edge)
            image = augmentations["image"]
            mask = augmentations["mask"]
            edge = augmentations["edge"]
        return image,mask,edge
#原版代码
class MyDataset(Dataset):
    def __init__(self ,image_dir , mask_dir, transform = None):


        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(self.image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.image_dir,self.images[index]) #image图片文件
        mask_path = os.path.join(self.mask_dir,self.images[index]) #mask图片文件 # 按需添加：segment_name.replace('png','jpg')

        image = np.array(Image.open(image_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"),dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image = image,mask = mask)
            image = augmentations["image"]
            mask = augmentations["mask"]
        return image,mask


In [8]:
# main()

In [9]:
def train_fn(loader, model, optimizer, scheduler,loss_fn, scaler,epoch):
    loop = tqdm(loader)

    for batch_idx, (data, targets,edges) in enumerate(loop):
    # for data, targets in loader:
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)
        edges = edges.float().unsqueeze(1).to(device=DEVICE)
#         print(targets.shape)#4 1 256 256
#         print(edges.shape)#4 1 256 256
        

        # forward
        with torch.cuda.amp.autocast():
#             predictions = model(data,edges)
            predictions = model(data)
            #保存一些图片来看看训练集的效果
#             torchvision.utils.save_image(predictions,f"./result/P_{epoch}.png")
            loss = loss_fn(predictions, targets)
            writer.add_scalar(" loss vs batch_idx",scalar_value=loss,global_step=batch_idx + epoch * (526//4))
#         writer.add_scalar(" loss vs epoch",scalar_value=loss,global_step=epoch)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
#         scheduler.step(loss)

        

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
            
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )
    
    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
#     loss_fn = two_BCE_Dice_loss()
#     loss_fn = BinaryDiceLoss()
    loss_fn = BCE_Dice_loss()
#     loss_fn = nn.BCEWithLogitsLoss()
#     loss_fn = DiceLoss()
#     optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    optimizer = optim.Adam([{'params':model.parameters()},{'params':loss_fn.a},{'params':loss_fn.b}], lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5, patience=scheduler_patience, verbose=True)
    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        TRAIN_EDGE_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("./my_checkpoint.pth.tar"), model)


    # check_accuracy(val_loader, model, device=DEVICE)
    # save_predictions_as_imgs(
    #         val_loader, model, folder="/content/gdrive/MyDrive/UNet/saved_images", device=DEVICE
    #     )
    
    scaler = torch.cuda.amp.GradScaler()
    best_test_score = 0
    best_epoch = 0
    best_checkpoint = None
    for epoch in range(NUM_EPOCHS):
        print(f"第{epoch + 1}轮训练开始！")
        train_fn(train_loader, model, optimizer, scheduler,loss_fn, scaler,epoch)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        #save_checkpoint(checkpoint)

        # check accuracy
        dice_score,test_score = check_accuracy(val_loader,model,device=DEVICE)
        
        writer.add_scalar("dice vs epoch",dice_score,epoch)
        writer.add_scalar("test_score vs epoch",test_score,epoch)
        
        #早停
        
        if test_score > best_test_score:
            best_test_score =test_score
            best_epoch = epoch
            best_checkpoint = checkpoint
        if epoch - best_epoch > P:
            save_checkpoint(best_checkpoint)
            print(f"早停机制触发，保存了第{best_epoch+1}轮的模型")
            break
        if epoch == NUM_EPOCHS -1:
            save_checkpoint(best_checkpoint)
            print("早停机制没有触发，但已保存最好的模型")


In [10]:
def save_checkpoint(state , filename='./my_checkpoint.pth.tar'):
    print('=> Saving checkpoint 模型已保存')
    torch.save(state , filename)
def load_checkpoint(checkpoint,model):
    print("=> Loading checkpoint 模型已读取")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
        train_dir,
        train_maskdir,
        train_edgedir,
        val_dir,
        val_maskdir,
        batch_size,
        train_transform,
        val_transform,
        num_workers = 4,
        pin_memory = True,

    ):
    train_dataset = MyDataset_train(
        image_dir = train_dir,
        mask_dir= train_maskdir,
        edge_dir = train_edgedir,
        transform=train_transform,

    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_dataset = MyDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader,val_loader

def check_accuracy(loader, model ,device = "cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x ,y in loader:
            
            
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            
            preds_nologits = model(x)

            preds = torch.sigmoid(preds_nologits)
            preds = (preds > 0.5).float()
            num_correct += (preds==y).sum()
            num_pixels += torch.numel(preds)
             
            dice_score += (2*(preds * y).sum())/(preds+y).sum() + 1e-8
    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Dice score : {dice_score/len(loader)}")
    model.train()
    return dice_score/len(loader) , num_correct/num_pixels
    

In [11]:
#通道注意力机制
class channel_attention(nn.Module):
    def __init__(self,channel,ratio = 16):
        super(channel_attention, self).__init__()
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.fc = nn.Sequential(
            nn.Linear(channel,channel//ratio,False),
            nn.ReLU(),
            nn.Linear(channel//ratio, channel , False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        b,c,h,w = x.size()
        max_pool_out = self.max_pool(x).view([b,c])
        avg_pool_out = self.avg_pool(x).view([b,c])

        max_fc_out = self.fc(max_pool_out)
        avg_fc_out = self.fc(avg_pool_out)

        out = max_fc_out + avg_fc_out
        out = self.sigmoid(out).view([b,c,1,1])

        return out *  x
    

#空间注意力机制
class spacial_attention(nn.Module):
    def __init__(self):
        super(spacial_attention, self).__init__()
        self.conv = nn.Conv2d(2,1,kernel_size=3,padding=1,bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        b,c,h,w = x.size()
#         print(type(x))
        max_pool_out ,_= (torch.max(x,dim=1,keepdim=True))
        avg_pool_out = (torch.mean(x,dim=1,keepdim=True))

        # print(type(max_pool_out))
        pool_out = torch.cat((max_pool_out,avg_pool_out),dim=1)
        out = self.conv(pool_out)
        out = self.sigmoid(out)

        return out * x

class Cbam(nn.Module):
    def __init__(self,channel , ratio = 16):
        super(Cbam, self).__init__()
        self.channel_attention = channel_attention(channel,ratio)
        self.spacial_attention = spacial_attention()

    def forward(self,x):
        b,c,h,w = x.size()
        x = self.channel_attention(x)
        x = self.spacial_attention(x)
        return x

In [12]:
class DoubleConv(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(DoubleConv,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,3,1,1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),


        )
    def forward(self,x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(self,in_channels = 3,out_channels= 1,features = [64,128,256,512]):
        super(UNET,self).__init__()
        self.ups = nn.ModuleList() 
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        


        
        #UNET 下采样部分
        for feature in features:
            self.downs.append(DoubleConv(in_channels,feature))
            in_channels = feature

        # UNET 上采样部分
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(feature*2,feature,kernel_size=2,stride=2), #0
            ) 
            self.ups.append(DoubleConv(feature*2,feature))                    #1
#             self.ups.append(nn.Conv2d(feature ,feature,kernel_size = 3,padding = 1,stride = 1))
            self.ups.append(Cbam(feature))                                    #2


        self.bottleneck = DoubleConv(features[-1],features[-1]*2)

        self.final_conv_1 = nn.Conv2d(features[0],out_channels,kernel_size=1)
#         self.final_conv_2 = nn.Conv2d(features[0],out_channels,kernel_size=1)
    def forward(self,x,*edge):

        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x  = self.bottleneck(x)

        skip_connections = skip_connections[::-1]
        
        #判断掩码是否存在
        flag = 0
        if edge == ():
            #不存在
            flag = 0
            pass
        else:
            #存在
            edge = edge[0]
            flag = 1
        
        #分成两条路
        x_1 = x

        
        #边缘增强的路
        
        for idx in range(0,len(self.ups),3):
            #进行反卷积
            x_1 = self.ups[idx](x_1)
            #获得下采样连接部分
            skip_connection = skip_connections[idx//3]
            #检查图片长宽是否符合要求
#             print("x.shape",x.shape)
#             print("skip_connection.shape",skip_connection.shape)
            if x_1.shape != skip_connection.shape:
                x_1 = TF.resize(x_1,size=skip_connection.shape[2:])

            #与下采样部分进行连接
            concat_skip = torch.cat((skip_connection,x_1),dim=1)

            b,c,h,w = x_1.size()
            
            if flag == 1:#如果掩膜存在 4,1,256,256
                edge_aft = edge.expand(-1,c * 2,-1,-1)
                edge_aft = torch.nn.functional.interpolate(edge_aft,h) #4,1024,32,32
#                 print("expand后",edge_aft.shape)
            if flag == 0:#如果掩膜不存在                
                edge_aft = concat_skip

            #和掩膜连接,或者与复制的自己连接
            x_1 = torch.cat([concat_skip,edge_aft],dim = 1)
#                 print("与掩膜cat后",x.shape)

            #进行两次卷积
            x_1 = self.ups[idx + 1](concat_skip)

            #经过通道和空间注意力模块
            x_1 = self.ups[idx + 2](x_1)
#                 print("cbam后",x.shape)

       
        return self.final_conv_1(x_1)

    
# 原版代码
class UNET(nn.Module):
    def __init__(self,in_channels = 3,out_channels= 1,features = [64,128,256,512]):
        super(UNET,self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)

        #UNET 下采样部分
        for feature in features:
            self.downs.append(DoubleConv(in_channels,feature))
            in_channels = feature

        # UNET 上采样部分
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(feature*2,feature,kernel_size=2,stride=2),
            )
            self.ups.append(DoubleConv(feature*2,feature))


        self.bottleneck = DoubleConv(features[-1],features[-1]*2)

        self.final_conv = nn.Conv2d(features[0],out_channels,kernel_size=1)
    def forward(self,x):
        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x  = self.bottleneck(x)

        skip_connections = skip_connections[::-1]

        for idx in range(0,len(self.ups),2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            #检查图片长宽是否符合要求
            if x.shape != skip_connection.shape:
                x = TF.resize(x,size=skip_connection.shape[2:])
            concat_skip = torch.cat((skip_connection,x),dim=1)
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)

def test():
    
    x = torch.randn((4,1,256,256))
    edge = torch.randn((4,1,256,256))
    
    model = UNET(in_channels=1,out_channels=1)
#     preds_1, preds_2= model(x,edge)
    preds = model(x,edge)
    print('输入的shape',x.shape)
    print('输出的shape',preds.shape)
#     print(preds_2.shape)
    assert preds.shape == x.shape

# if __name__ == '__main__':
#     test()

In [13]:
def flatten(tensor):
    """Flattens a given tensor such that the channel axis is first.
    The shapes are transformed as follows:
       (N, C, D, H, W) -> (C, N * D * H * W)
    """
    
    C = tensor.size(1)        #获得图像的维数
    # new axis order
    axis_order = (1, 0) + tuple(range(2, tensor.dim()))     
    # Transpose: (N, C, D, H, W) -> (C, N, D, H, W)
    transposed = tensor.permute(axis_order)                 #将维数的数据转换到第一位
    # Flatten: (C, N, D, H, W) -> (C, N * D * H * W)
    return transposed.contiguous().view(C, -1)              


class DiceLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.epsilon = 1e-5

    def forward(self, output, target):
        assert output.size() == target.size(), "'input' and 'target' must have the same shape"
#         output = F.softmax(output, dim=1)
        output = torch.sigmoid(output)
        output = flatten(output)
        target = flatten(target)
        # intersect = (output * target).sum(-1).sum() + self.epsilon
        # denominator = ((output + target).sum(-1)).sum() + self.epsilon

        intersect = (output * target).sum(-1)
        denominator = (output + target).sum(-1)
        dice = intersect / denominator
        dice = torch.mean(dice)
        return 1 - dice
    
# class BinaryDiceLoss(nn.Module):
#     def __init__(self):
#         super(BinaryDiceLoss, self).__init__()

#     def forward(self, input, targets):
#         # 获取每个批次的大小 N
#         input = torch.sigmoid(input)
#         N = targets.size()[0]
#         # 平滑变量
#         smooth = 1
#         # 将宽高 reshape 到同一纬度
#         input_flat = input.view(N, -1)
#         targets_flat = targets.view(N, -1)

#         # 计算交集
#         intersection = input_flat * targets_flat
#         N_dice_eff = (2 * intersection.sum(1) + smooth) / (input_flat.sum(1) + targets_flat.sum(1) + smooth)
#         # 计算一个批次中平均每张图的损失
#         loss = 1 - N_dice_eff.sum() / N
#         return loss

class BCE_Dice_loss(nn.Module):
    def __init__(self):
        super(BCE_Dice_loss,self).__init__()
        
        self.a = torch.rand(1).to(device=DEVICE)
        self.b = torch.rand(1).to(device=DEVICE)
        
#         self.dice_loss = BinaryDiceLoss()
        self.dice_loss = DiceLoss()
        self.BCE_loss = nn.BCEWithLogitsLoss()  #bceloss
    def forward(self,input,target):
        
        return self.b*self.dice_loss(input,target) + self.a *self.BCE_loss(input,target)
#         return (1 - 0.6)*self.dice_loss(input,target) + 0.6*self.BCE_loss(input,target)

# class two_BCE_Dice_loss(nn.Module):
#     def __init__(self):
#         super(two_BCE_Dice_loss,self).__init__()
#         self.BCE_Dice_loss_1 = BCE_Dice_loss()
#         self.BCE_Dice_loss_2 = BCE_Dice_loss()
#     def forward(self,input_1,input_2,target):
#         return 0.25 * self.BCE_Dice_loss_1(input_1,target) + 0.65 * self.BCE_Dice_loss_2(input_2,target)



In [14]:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
NUM_EPOCHS = 65
NUM_WORKERS = 2
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
PIN_MEMORY = True
LOAD_MODEL = False
# LOAD_MODEL = True
TRAIN_IMG_DIR = "../input/pre-unet/pre-UNet/dataset/train/images"
TRAIN_MASK_DIR = "../input/pre-unet/pre-UNet/dataset/train/labels"
TRAIN_EDGE_DIR = "../input/pre-unet/pre-UNet/dataset/train/masks"
VAL_IMG_DIR = "../input/pre-unet/pre-UNet/dataset/test/images"
VAL_MASK_DIR = "../input/pre-unet/pre-UNet/dataset/test/labels"
P = 6
scheduler_patience = 50


In [38]:
writer = SummaryWriter("./")
main()

第1轮训练开始！


100%|██████████| 132/132 [00:23<00:00,  5.60it/s, loss=0.674]


Got 17996738/18284544 with acc 98.43
Dice score : 0.6040681600570679
第2轮训练开始！


100%|██████████| 132/132 [00:23<00:00,  5.62it/s, loss=0.607]


Got 18158206/18284544 with acc 99.31
Dice score : 0.8110133409500122
第3轮训练开始！


...

Got 18239644/18284544 with acc 99.75
Dice score : 0.9231589436531067
第45轮训练开始！


100%|██████████| 132/132 [00:23<00:00,  5.64it/s, loss=0.279]


Got 18244250/18284544 with acc 99.78
Dice score : 0.9298909902572632
第46轮训练开始！


100%|██████████| 132/132 [00:23<00:00,  5.63it/s, loss=0.298]


Got 18238795/18284544 with acc 99.75
Dice score : 0.9279106259346008
=> Saving checkpoint 模型已保存
早停机制触发，保存了第39轮的模型


In [37]:
import os
os.listdir("./")
# os.makedirs('./result')
# os.remove("./events.out.tfevents.1650290892.653858dba76a.33.2")
# os.remove("./my_checkpoint.pth.tar")

# 删除./下的所有图片
# _list = os.listdir('./')
# for i in _list:
#     os.remove(os.path.join('./',i))

# 删除result下的所有图片
# result_list = os.listdir('./result')
# for i in result_list:
#     os.remove(os.path.join('./result',i))


In [39]:
import cv2
import torch
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
transform = transforms.Compose([
    transforms.ToTensor()
])

net = UNET().to(device = DEVICE)

# load_checkpoint(torch.load("../input/model/my_checkpoint.pth_1.tar"),net)
load_checkpoint(torch.load("./my_checkpoint.pth.tar"),net)
labels_list = os.listdir('../input/pre-unet/pre-UNet/dataset/test/labels')
net.eval()
with torch.no_grad():
    for i in range(len(labels_list)):
        img = cv2.imread(os.path.join('../input/pre-unet/pre-UNet/dataset/test/images',labels_list[i]))

        output = net(transform(img).unsqueeze(0).to(device = DEVICE))
        save_image(output, os.path.join('./result',labels_list[i]))

    #     temp = np.array(Image.open(os.path.join('./result',labels_list[i])).convert("L"),dtype=np.float32)
    #     temp[temp == 1.0] = 255.0
    #     cv2.imwrite(os.path.join('./result',labels_list[i]), temp)
    #     if i == 10 :
    #         break
        if i % 10 == 0:
            print(f"已处理{i+1}张图片")
    #         break
        if i == len(labels_list) -1:
            print(f"一共处理{i+1}张图片，完成！")

=> Loading checkpoint 模型已读取
已处理1张图片
已处理11张图片
已处理21张图片
已处理31张图片
已处理41张图片
已处理51张图片
已处理61张图片
已处理71张图片
已处理81张图片
已处理91张图片
已处理101张图片
已处理111张图片
已处理121张图片
已处理131张图片
已处理141张图片
已处理151张图片
已处理161张图片
已处理171张图片
已处理181张图片
已处理191张图片
已处理201张图片
已处理211张图片
已处理221张图片
已处理231张图片
已处理241张图片
已处理251张图片
已处理261张图片
已处理271张图片
一共处理279张图片，完成！


In [21]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
gt_path = '../input/pre-unet/pre-UNet/dataset/test/labels'
res_path = './result'#真正运行时用这个

In [ ]:
temp = np.array(Image.open('./result/SC-HF-I-09-IM-0001-0227.dcm.png').convert("L"),dtype=np.float32)
print(temp)


In [40]:
import math
def F(res_path, gt_path):
#     gt_path = '../input/pre-unet/pre-UNet/dataset/test/labels'
#     res_path = './result'#真正运行时用这个
    res_list = os.listdir(res_path)
#     print(res_list)
    P = [0 for _ in range(len(res_list))]
#     print(len(P))
    R = [0 for _ in range(len(res_list))]
    PA = [0 for _ in range(len(res_list))]


    for i in range(len(res_list)):
        lam = 1e-8
        r_name = res_path + '/' + res_list[i]
        
        g_name = gt_path + '/' + res_list[i]
        res = cv2.imread(r_name)
        res = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY)
        h, w = res.shape

        gt = cv2.imread(g_name)
        gt = cv2.cvtColor(gt, cv2.COLOR_RGB2GRAY)
        _, gt = cv2.threshold(gt, 125, 255, cv2.THRESH_BINARY)
#         print(i, r_name)
        th = 2 * sum(sum(res)) / (h * w)  ##阈值
        _, res_tmp = cv2.threshold(res, th, 255, cv2.THRESH_BINARY)
        tmp = res_tmp - gt
        FP = sum(sum(tmp == 255))
        FN = sum(sum(tmp == -255))
#         FN = sum(sum(tmp == 0))
        TP = sum(sum((res_tmp == gt) & (gt == 255)))
        TN = sum(sum((res_tmp == gt) & (gt == -255)))

        
        # acc = (TP + TN) / (TP + TN + FP + TN)
        PA[i] = (TP + TN) / (TP + TN + FP + TN)


        P[i] = TP / (TP + FP) + lam
        R[i] = TP / (TP + FN) + lam
    P = [x for x in P if math.isnan(x) == False]
    R = [x for x in R if math.isnan(x) == False]

    belt2 = 0.3
    
    p = sum(P) / len(P)
    r = sum(R) / len(R)
    pa = sum(PA) / len(PA)
    Fmeasure = ((1 + belt2) * p * r) / (belt2 * p + r)
    # print(Fmeasure)
    return p, r, Fmeasure
    # return pa

# print(F(res_path, gt_path)) ##p=0.863,r=1,F=0.893


def dice(res_path, gt_path):
    # res_path = 'D:/2020_Work/20200306_Trash_class/Semantic/res/ResNet50_100/'
    # gt_path = 'D:/2020_Work/20200306_Trash_class/Semantic/data/test/labels/'
    res_list = os.listdir(res_path)
    di = []
    for i in range(len(res_list)):
        r_name = res_path + '/' +res_list[i]
        g_name = gt_path + '/' +res_list[i]
        res = cv2.imread(r_name)
        h, w, _ = res.shape
        res = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY)
        _,res = cv2.threshold(res, 125, 255, cv2.THRESH_BINARY)

        gt = cv2.imread(g_name)
        gt = cv2.cvtColor(gt, cv2.COLOR_RGB2GRAY)
        _, gt = cv2.threshold(gt, 125, 255, cv2.THRESH_BINARY)

        iou_and = sum(sum((res == 255) & (gt == 255)))
        iou_or = sum(sum(res == 255)) + sum(sum(gt == 255))
#         print('sum(res == 255)',sum(res == 255))
#         print('sum(sum(res == 255))',sum(sum(res == 255)))
        
#         print(2 * iou_and, iou_or)
        
#         preds = (res > 1)
#         y = (gt > 1)
#         iou_and = (preds * y).sum()
#         iou_or = (preds+y).sum()
#         print(2 * iou_and, iou_or)
        

        
        di.append(2 * iou_and / iou_or)

    return sum(di) / len(di)


# print('dice=',dice(res_path ,gt_path)) ##dice= 0.9688


def Hausdorff(res_path, gt_path):
    res_list = os.listdir(res_path)
    haus = []
    for i in range(len(res_list)):
        r_name = res_path + '/' +res_list[i]
        g_name = gt_path + '/' +res_list[i]
#         print(r_name, g_name)
        res = cv2.imread(r_name)
        h, w, _ = res.shape

        gt = cv2.imread(g_name)

        # 2.获取图片连通域
        # 灰度化, 二值化, 连通域分析
        
        img_gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        ret, img_bin = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
        contours1, hierarchy = cv2.findContours(img_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        
        img_gray = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)
        ret, img_bin = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
        contours2, hierarchy = cv2.findContours(img_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        
#         print(type(contours1))
#         print(len(contours1))
        if len(contours1) == 0 or len(contours2) == 0:
            continue
        cnt_cs1 = contours1[0]
        cnt_cs2 = contours2[0]

        # 3.创建计算距离对象
        hausdorff_sd = cv2.createHausdorffDistanceExtractor()

        # 4.计算轮廓之间的距离
        d1 = hausdorff_sd.computeDistance(cnt_cs1, cnt_cs2)
        haus.append(d1)
    return sum(haus) / len(haus)
#         return sum(haus)

# 仅需要F_1,mean_dice,Hausdorff
# 既Hausdorff(),dice(),F()


F = F(res_path, gt_path)
Hausdorff = Hausdorff(res_path, gt_path)
dice = dice(res_path, gt_path)
print("p=", F[0])
print("r=", F[1])
print("F1=",F[2])
print("dice=", dice)
print("Hausdorff=", Hausdorff)  # Hausdorff= 65.96012181915597

#p= 0.9309735800459854
# r= 1.000000010000003
# F1= 0.9460432697787156
# dice= 0.892990253079993
# Hausdorff= 2.9093599520998894

# p= 0.9525923600557417
# r= 1.000000010000003
# F1= 0.9631292120743221
# dice= 0.8633607913889234
# Hausdorff= 3.2504405320774423

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in long_scalars
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in long_scalars
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in long_scalars


p= 0.8992816607674043
r= 1.000000010000003
F1= 0.9206807636342084
dice= 0.901683515889191
Hausdorff= 5.228526370248932


In [ ]:
def get_contours(img):
    """获取连通域

    :param img: 输入图片
    :return: 最大连通域
    """
#     print(img.size)#196608
#     print(img.shape)#(256,256,3)

    # 灰度化, 二值化, 连通域分析
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_bin = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours[0]

def Hausdorff(res_path, gt_path):
    res_list = os.listdir(res_path)
    haus = []
    for i in range(len(res_list)):
        r_name = res_path + '/'+res_list[i]
        g_name = gt_path +  '/'+res_list[i]
#         print(r_name, g_name)
        res = cv2.imread(r_name)
        h, w, _ = res.shape

        gt = cv2.imread(g_name)

        # 2.获取图片连通域
        cnt_cs1 = get_contours(res)
        cnt_cs2 = get_contours(gt)

        # 3.创建计算距离对象
        hausdorff_sd = cv2.createHausdorffDistanceExtractor()

        # 4.计算轮廓之间的距离
        d1 = hausdorff_sd.computeDistance(cnt_cs1, cnt_cs2)
        haus.append(d1)
    return sum(haus) / len(haus)


# def main():

#     # 1.导入图片
#     img_cs1 = cv2.imread("./result/SC-HF-NI-14-IM-0001-0103.dcm.png")
#     img_cs2 = cv2.imread("./result/SC-HYP-12-IM-0001-0140.dcm.png")
#     img_hand = cv2.imread("./result/SC-HYP-12-IM-0001-0120.dcm.png")

#     # 2.获取图片连通域
#     cnt_cs1 = get_contours(img_cs1)
#     cnt_cs2 = get_contours(img_cs2)
#     cnt_hand = get_contours(img_hand)

#     # 3.创建计算距离对象
#     hausdorff_sd = cv2.createHausdorffDistanceExtractor()

#     # 4.计算轮廓之间的距离
#     d1 = hausdorff_sd.computeDistance(cnt_hand, cnt_hand)
#     print("与自身的距离hausdorff\t d1=", d1)

#     d2 = hausdorff_sd.computeDistance(cnt_hand, cnt_cs2)
#     print("与相似图片的距离hausdorff\t d2=", d2)

#     d3 = hausdorff_sd.computeDistance(cnt_hand, cnt_cs1)
#     print("与不同图片的距离hausdorff\t d3=", d3)

#     # 5.显示图片
# main()